<a href="https://colab.research.google.com/github/shelly-hsu/A-style-transfer-sevice/blob/main/style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from __future__ import print_function
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse
import tensorflow as tf
from keras.applications import vgg19
from keras import backend as K
from keras.preprocessing.image import load_img
import cv2
from google.colab import files
tf.compat.v1.disable_eager_execution()

def style_transfer():
  output_folder_path = "/content/drive/MyDrive/output_img"     //the folder we store genrated image
  result_prefix = "output"
  iterations = 10                         // iteration can be chage if you want the result to more obvious

  # 原圖與風格圖佔output比重
  content_weight = 0.025
  style_weight = 1.0
  # 損失總差異預設值
  total_variation_weight = 1.0

  # output 圖的寬高
  width, height = pre_content.size
  img_nrows = 400
  img_ncols = int(width * img_nrows / height)

  # 轉換成 VGG 19 input 格式
  def preprocess_image(image):
      img = image.resize((img_ncols, img_nrows))
      img = img_to_array(img)
      img = np.expand_dims(img, axis=0)
      img = vgg19.preprocess_input(img)
      return img

  # 將特徵向量轉換成 image
  def deprocess_image(x):
      if K.image_data_format() == 'channels_first':
          x = x.reshape((3, img_nrows, img_ncols))
          x = x.transpose((1, 2, 0))
      else:
          x = x.reshape((img_nrows, img_ncols, 3))
      # 設定RGB顏色的中心點 (Remove zero-center by mean pixel)
      x[:, :, 0] += 103.939
      x[:, :, 1] += 116.779
      x[:, :, 2] += 123.68
      # 'BGR'->'RGB'
      x = x[:, :, ::-1]
      x = np.clip(x, 0, 255).astype('uint8')
      return x

  # 設定 Keras 變數 base_image = 原圖 向量
  base_image = K.variable(preprocess_image(pre_content))
  # 設定 Keras 變數 style_reference_image = 風格圖 向量
  style_reference_image = K.variable(preprocess_image(pre_style))

  # 設定合成圖的起始值
  if K.image_data_format() == 'channels_first':
      combination_image = K.placeholder((1, 3, img_nrows, img_ncols))
  else:
      combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

  # 合併原圖、風格圖、合成圖 向量
  input_tensor = K.concatenate([base_image,style_reference_image,combination_image], axis=0)

  # 載入 VGG 19 模型，不包括加在最後3層的卷積層
  model = vgg19.VGG19(input_tensor=input_tensor,weights='imagenet', include_top=False)

  # get the symbolic outputs of each "key" layer (we gave them unique names).
  # 讀取 VGG 19 模型的每一層的名稱與output
  outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

  # 計算 風格 loss 的 gram matrix
  def gram_matrix(x):
      if K.image_data_format() == 'channels_first':
          features = K.batch_flatten(x)
      else:
          features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
      gram = K.dot(features, K.transpose(features))
      return gram

  # 計算 風格 loss 
  def style_loss(style, combination):
      assert K.ndim(style) == 3
      assert K.ndim(combination) == 3
      S = gram_matrix(style)
      C = gram_matrix(combination)
      channels = 3
      size = img_nrows * img_ncols
      return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

  # 計算 content loss 
  def content_loss(base, combination):
      return K.sum(K.square(combination - base))

  # the 3rd loss function, total variation loss,
  # designed to keep the generated image locally coherent


  # 計算 損失總差異(total variation loss)，以利合成圖的連貫性
  def total_variation_loss(x):
      assert K.ndim(x) == 4
      if K.image_data_format() == 'channels_first':
          a = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
          b = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
      else:
          a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
          b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
      return K.sum(K.pow(a + b, 1.25))

  # 彙總上面三項的損失(loss)
  loss = K.variable(0.)
  layer_features = outputs_dict['block5_conv2']
  base_image_features = layer_features[0, :, :, :]
  combination_features = layer_features[2, :, :, :]
  loss = loss + content_weight * content_loss(base_image_features,combination_features)

  feature_layers = ['block1_conv1', 'block2_conv1',
                    'block3_conv1', 'block4_conv1',
                    'block5_conv1']
  for layer_name in feature_layers:
      layer_features = outputs_dict[layer_name]
      style_reference_features = layer_features[1, :, :, :]
      combination_features = layer_features[2, :, :, :]
      sl = style_loss(style_reference_features, combination_features)
      loss += (style_weight / len(feature_layers)) * sl
  loss += total_variation_weight * total_variation_loss(combination_image)

  # 計算合成圖的梯度(gradients)
  grads = K.gradients(loss, combination_image)

  # 建立 Keras function API 模型
  outputs = [loss]
  if isinstance(grads, (list, tuple)):
      outputs += grads
  else:
      outputs.append(grads)

  f_outputs = K.function([combination_image], outputs)


  # 依梯度下降法，評估模型
  def eval_loss_and_grads(x):
      if K.image_data_format() == 'channels_first':
          x = x.reshape((1, 3, img_nrows, img_ncols))
      else:
          x = x.reshape((1, img_nrows, img_ncols, 3))
      outs = f_outputs([x])
      loss_value = outs[0]
      if len(outs[1:]) == 1:
          grad_values = outs[1].flatten().astype('float64')
      else:
          grad_values = np.array(outs[1:]).flatten().astype('float64')
      return loss_value, grad_values

  # 評估模型類別
  # this Evaluator class makes it possible
  # to compute loss and gradients in one pass
  # while retrieving them via two separate functions,
  # "loss" and "grads". This is done because scipy.optimize
  # requires separate functions for loss and gradients,
  # but computing them separately would be inefficient.
  class Evaluator(object):
      def __init__(self):
          self.loss_value = None
          self.grads_values = None

      def loss(self, x):
          assert self.loss_value is None
          loss_value, grad_values = eval_loss_and_grads(x)
          self.loss_value = loss_value
          self.grad_values = grad_values
          return self.loss_value

      def grads(self, x):
          assert self.loss_value is not None
          grad_values = np.copy(self.grad_values)
          self.loss_value = None
          self.grad_values = None
          return grad_values

  # 執行模型評估
  evaluator = Evaluator()

  # run scipy-based optimization (L-BFGS) over the pixels of the generated image
  # so as to minimize the neural style loss
  x = preprocess_image(pre_content)

  # 在每一週期產生合成圖
  for i in range(iterations):
      print('Start of iteration', i)
      start_time = time.time()
      x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20)
      print('Current loss value:', min_val)
      # 儲存每一週期的output合成圖
      img = deprocess_image(x.copy())
      fname = result_prefix + '_at_iteration_%d.png' % i
      print("store")
      cv2.imwrite(os.path.join(output_folder_path , fname), img)
      end_time = time.time()
      print('Image saved as', fname)
      print('Iteration %d completed in %ds' % (i, end_time - start_time))
    
      

In [ ]:
import io
import anvil.server
import anvil.media
from google.colab.patches import cv2_imshow
from anvil import URLMedia
from anvil.google.drive import app_files


anvil.server.connect("DISTCQZ2DFDQ5D4HJJFCU7PU-2YNRRVOF3GPXB4K2")
output_folder_path = "/content/drive/MyDrive/output_img"
@anvil.server.callable
def get_input_img(file):
  global pre_content
  with anvil.media.TempFile(file) as f:
    pre_content = load_img(f)
@anvil.server.callable
def get_style_img(file):
  global pre_style
  with anvil.media.TempFile(file) as f:
    pre_style = load_img(f)

@anvil.server.callable
def getimage():
  img = load_img("/content/drive/MyDrive/output_img/output_at_iteration_9.png")
  img_byte_arr = io.BytesIO()
  img.save(img_byte_arr, format='JPEG')
  img_byte_arr = img_byte_arr.getvalue()
  media_obj = anvil.BlobMedia(content_type="image/jpeg", content=img_byte_arr)
  return media_obj
  
@anvil.server.callable
def start_style_transfer():
  style_transfer()
  print("finish")
  
anvil.server.wait_forever()

Start of iteration 0
Current loss value: 20975272000.0
store
Image saved as output_at_iteration_0.png
Iteration 0 completed in 13s
Start of iteration 1
Current loss value: 11533148000.0
store
Image saved as output_at_iteration_1.png
Iteration 1 completed in 12s
Start of iteration 2
Current loss value: 8613443000.0
store
Image saved as output_at_iteration_2.png
Iteration 2 completed in 12s
Start of iteration 3
Current loss value: 7131065300.0
store
Image saved as output_at_iteration_3.png
Iteration 3 completed in 12s
Start of iteration 4
Current loss value: 6259508700.0
store
Image saved as output_at_iteration_4.png
Iteration 4 completed in 12s
Start of iteration 5
Current loss value: 5660512000.0
store
Image saved as output_at_iteration_5.png
Iteration 5 completed in 12s
Start of iteration 6
Current loss value: 5254630400.0
store
Image saved as output_at_iteration_6.png
Iteration 6 completed in 12s
Start of iteration 7
Current loss value: 4983628000.0
store
Image saved as output_at_ite

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pymysql

In [ ]:
!pip install -U scipy==1.2.0

     |████████████████████████████████| 26.6 MB 46.3 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.2.25 requires scipy>=1.2.1, but you have scipy 1.2.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip install anvil-uplink

     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 51 kB 217 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=04640e70995950367e11d803ebcde56e10e4eb90f2b0940f7fee44602075fa61
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py
